In [9]:
import numpy as np
import pywt

# custom libs
import sys
sys.path.append("../..")
from src.dynamicFPC import L2norm, inner_product
from src.simulations import simulate_curves, generate_coeffs

# KdFPC

In [ ]:
def KdFPC_bootstrap(Y, B, lag_max, thetahat_old, gammahat_old, Ydev, m, du, Ybar, n, p, alpha):
    bs_pvalues = np.zeros(lag_max)

    print(thetahat_old)
    for d0 in range(1, lag_max + 1):

        # ----------------------------------------------------
        # H0 eigenvalue
        # ----------------------------------------------------
        thetahatH0 = np.real(thetahat_old[d0])
        # print('thetahatH0', thetahatH0)

        # ----------------------------------------------------
        # Step 4 logic (restricted to dimension d0)
        # ----------------------------------------------------
        gammahat = np.real(gammahat_old[:, :d0])

        psihat_root = np.dot(Ydev[:, :(n - p)], gammahat)

        psihat = np.zeros((m, d0))
        for i in range(d0):
            psihat[:, i] = psihat_root[:, i] / L2norm(psihat_root[:, i], du)

        etahat = inner_product(psihat, Ydev, du)
        Yhat = Ybar + np.dot(psihat, etahat)

        epsilonhat = Y - Yhat

        # ----------------------------------------------------
        # Bootstrap
        # ----------------------------------------------------
        def sampleCols(A):
            return A[:, np.random.permutation(A.shape[1])]

        bs_thetahat = np.zeros(B)

        for b in range(B):
            # Bootstrap sample
            bs_Y = Yhat + sampleCols(epsilonhat)

            bs_Ybar = np.mean(bs_Y, axis=1, keepdims=True)
            bs_Ydev = bs_Y - bs_Ybar

            # Rebuild K*
            bs_core = inner_product(bs_Ydev, bs_Ydev, du)
            bs_Kstar_core0 = bs_core[:(n - p), :(n - p)]

            bs_Kstar_core = np.zeros((n - p, n - p, p))
            for k in range(1, p + 1):
                bs_Kstar_core[:, :, k - 1] = bs_core[
                    k:(n - (p - k)),
                    k:(n - (p - k))
                ]

            bs_Kstar_sum = np.sum(bs_Kstar_core, axis=2)
            bs_Kstar = (n - p) ** (-2) * np.dot(bs_Kstar_sum, bs_Kstar_core0)

            # Eigenvalues
            bs_eigvals = np.linalg.eigvals(bs_Kstar)
            bs_eigvals = np.sort(np.real(bs_eigvals))[::-1]

            # print('bs_eigvals', bs_eigvals)
            # print('bs_eigvals[:d0]',bs_eigvals[:5])
            # print('bs_eigvals[d0]', bs_eigvals[d0])
            bs_thetahat[b] = bs_eigvals[d0]

        # ----------------------------------------------------
        # p-value
        # ----------------------------------------------------
        bs_pvalues[d0 - 1] = np.mean(bs_thetahat >= thetahatH0)
        # print(bs_pvalues)

    # --------------------------------------------------------
    # Selected dimension
    # --------------------------------------------------------
    idx = np.where(bs_pvalues < alpha)[0]
    # print("\n",idx)
    d0 = idx[0] + 1 if len(idx) > 0 else 1
    return d0

In [169]:
def KdFPC(Y, lag_max, B, alpha, du, p, m, u,
        select_ncomp=False, dimension=None):
    """
    Fit the dynamic KLE model.

    Parameters
    ----------
    lag_max : int
        Maximum number of components to test under bootstrapping.
    
    B : int
        Number of bootstrap replications.

    alpha : float
        Significance level for component selection.

    du : float
        Grid spacing in the domain of the curves.

    p : int
        Time-lag order used to form K*.

    m : int
        Number of grid points (redundant, but kept for compatibility).

    u : ndarray (m,)
        Grid support.

    select_ncomp : bool, optional (default=False)
        Whether to run the bootstrap procedure to determine d0.

    dimension : int, optional
        Fixed number of components to extract (used only if select_ncomp=False).

    Returns
    -------
    self : K_dFPC
        The fitted model.
    """
    n = N = Y.shape[1]

    # Mean and deviations
    Ybar = np.mean(Y, axis=1, keepdims=True)
    Ydev = Y - Ybar

    # --------------------------------------------------------
    # Step 1 — Build core matrices
    # --------------------------------------------------------

    core = inner_product(Ydev, Ydev, du)
    Kstar_core0 = core[:(n - p), :(n - p)]

    Kstar_core = np.zeros((n - p, n - p, p))
    for k in range(1, p + 1):
        Kstar_core[:, :, k - 1] = core[k:(n - (p - k)), k:(n - (p - k))]

    Kstar_sum = np.sum(Kstar_core, axis=2)
    Kstar = (n - p) ** (-2) * np.dot(Kstar_sum, Kstar_core0)

    # --------------------------------------------------------
    # Step 2 — Eigen-decomposition
    # --------------------------------------------------------

    eigvals, eigvecs = np.linalg.eig(Kstar)
    eigvals = np.real(eigvals)
    eigvecs = np.real(eigvecs)

    # Sort descending
    idx = np.argsort(eigvals)[::-1]
    thetahat_old = eigvals[idx]
    gammahat_old = eigvecs[:, idx]

    # --------------------------------------------------------
    # Step 3 — Select number of components (optional bootstrap)
    # --------------------------------------------------------

    if select_ncomp:
        d0 = KdFPC_bootstrap(Y, B, lag_max, thetahat_old,gammahat_old,Ydev, m, du,Ybar,n,p, alpha)
    else:
        d0 = dimension

    return d0 
    # --------------------------------------------------------
    # Step 4 — Final estimation
    # --------------------------------------------------------

    # thetahat = thetahat_old[:d0]
    # gammahat = gammahat_old[:, :d0]

    # psihat_root = np.dot(Ydev[:, :(n - p)], gammahat)

    # psihat = np.zeros((m, d0))
    # for i in range(d0):
    #     psihat[:, i] = psihat_root[:, i] / L2norm(psihat_root[:, i], du)

    # etahat = inner_product(psihat, Ydev, du)
    # Yhat = Ybar + np.dot(psihat, etahat)
    # epsilonhat = Y - Yhat

In [170]:
# if select_ncomp:

#     # --------------------------------------------------------
#     # Bootstrap p-values for component selection
#     # --------------------------------------------------------

#     bs_pvalues = np.zeros(lag_max)

#     # Loop over candidate dimensions d0 = 1, ..., lag_max
#     for d0 in range(1, lag_max + 1):

#         # (d0+1)-th eigenvalue under H0
#         thetahatH0 = np.real(thetahat_old[d0])

#         # First d0 eigencomponents
#         thetahat = np.real(thetahat_old[:d0])
#         gammahat = np.real(gammahat_old[:, :d0])

#         # Build estimated functions
#         psihat_root = np.dot(Ydev[:, :(n - p)], gammahat)

#         psihat = np.zeros((m, d0))
#         for i in range(d0):
#             psihat[:, i] = psihat_root[:, i] / L2norm(psihat_root[:, i], du)

#         etahat = inner_product(psihat, Ydev, du)
#         Yhat = Ybar + np.dot(psihat, etahat)

#         # Enforce density constraints
#         Yhat_fix = Yhat.copy()
#         Yhat_fix[Yhat_fix < 0] = 0.0
#         for t in range(N):
#             Yhat_fix[:, t] /= np.sum(Yhat_fix[:, t]) * du

#         epsilonhat = Y - Yhat_fix

#         # ----------------------------------------------------
#         # Bootstrap procedure
#         # ----------------------------------------------------

#         def sample_cols(A):
#             """Resample columns with replacement."""
#             idx = np.random.randint(0, A.shape[1], size=A.shape[1])
#             return A[:, idx]

#         bs_thetahat = np.zeros(B)

#         for b in range(B):

#             # Bootstrap pseudo-observations
#             bs_epsilon = sample_cols(epsilonhat)
#             bs_Y = Yhat_fix + bs_epsilon

#             bs_Ybar = np.mean(bs_Y, axis=1, keepdims=True)
#             bs_Ydev = bs_Y - bs_Ybar

#             # Rebuild K*
#             bs_core = inner_product(bs_Ydev, bs_Ydev, du)
#             bs_Kstar_core0 = bs_core[:(n - p), :(n - p)]

#             bs_Kstar_core = np.zeros((n - p, n - p, p))
#             for k in range(1, p + 1):
#                 bs_Kstar_core[:, :, k - 1] = bs_core[
#                     k:(n - (p - k)),
#                     k:(n - (p - k))
#                 ]

#             bs_Kstar_sum = np.sum(bs_Kstar_core, axis=2)
#             bs_Kstar = (n - p) ** (-2) * np.dot(bs_Kstar_sum, bs_Kstar_core0)

#             # Eigenvalues
#             bs_eigvals = np.linalg.eigvals(bs_Kstar)
#             bs_eigvals = np.sort(np.real(bs_eigvals))[::-1]

#             # Store (d0+1)-th eigenvalue
#             bs_thetahat[b] = bs_eigvals[d0]

#         # Bootstrap p-value
#         bs_pvalues[d0 - 1] = np.mean(bs_thetahat >= thetahatH0)

#     # --------------------------------------------------------
#     # Selected dimension
#     # --------------------------------------------------------

#     idx = np.where(bs_pvalues < alpha)[0]
#     d0 = idx[0] + 2 if len(idx) > 0 else 1

In [171]:
n = 300          # sample size (curves)
d = 4           # dimension parameter
nt = 256         # number of grid points
u = np.linspace(0.01, 0.99, nt)[:, None]  # nt x 1 grid
phis = generate_coeffs(d) # AR(1) model coefficients
variance = 1.5

Y, X, mEps = simulate_curves(
                        n,
                        nt,
                        u,
                        phis, 
                        variances=np.full(len(phis), variance)
                        )

In [172]:
KdFPC_kwargs = {
    "lag_max": 5,
    "alpha": 0.2,
    "du": 0.05,
    "B": 1000,
    "p": 5,
    "m": nt,
    "u": u,
    "select_ncomp": True,
    "dimension": d,
}

In [173]:
KdFPC(Y, **KdFPC_kwargs)

[ 1.11708671e+04  1.42408643e+03  4.83329550e+02  1.28992171e+02
  1.40121321e+01  2.27561312e-01  4.85221703e-02  1.23517914e-02
  3.57471617e-03  6.76392580e-04  3.86657944e-04  4.28324988e-05
  1.26926496e-05  2.09306027e-06  1.43607848e-12  1.43607848e-12
  8.73965330e-13  8.73965330e-13  8.52203283e-13  8.52203283e-13
  4.93494617e-13  4.93494617e-13  4.04428616e-13  4.04428616e-13
  4.00567092e-13  3.77071132e-13  3.77071132e-13  3.33257184e-13
  3.33257184e-13  2.89281181e-13  2.89281181e-13  2.86354658e-13
  2.86354658e-13  2.79204911e-13  2.79204911e-13  2.75676374e-13
  2.71091773e-13  2.71091773e-13  2.57741231e-13  2.57741231e-13
  2.42582375e-13  2.42582375e-13  2.33726724e-13  2.24736063e-13
  2.24736063e-13  1.97650308e-13  1.97650308e-13  1.86264211e-13
  1.86264211e-13  1.85484143e-13  1.85484143e-13  1.57162172e-13
  1.57162172e-13  1.53458034e-13  1.53458034e-13  1.51686934e-13
  1.47817649e-13  1.47249630e-13  1.47249630e-13  1.37551435e-13
  1.37551435e-13  1.36190

np.int64(1)

# WdFPC

In [ ]:
def DimEst_boot(
    Y: np.ndarray,
    NREP: int,
    B: np.ndarray,
    p: int,
    N: int,
    wavelet: str
):
    """
    Bootstrap-based dimension estimation for wavelet dFPC.

    Parameters
    ----------
    Y : np.ndarray (nt, n)
        Functional observations (columns = time).
    NREP : int
        Number of bootstrap replications.
    B : np.ndarray (J, d0)
        Eigenvectors from original estimation.
    p : int
        Maximum lag.
    N : int
        Wavelet decomposition level.
    wavelet : str
        PyWavelets wavelet name (e.g. 'db2').

    Returns
    -------
    vd0p1_boot : np.ndarray (NREP,)
        Bootstrap estimates of the (d0 + 1)-th eigenvalue.
    """

    nt, n = Y.shape
    J, d0 = B.shape

    vd0p1_boot = np.zeros(NREP)

    # ----- wavelet decomposition of original data -----
    coeffs = pywt.wavedec(Y[:, 0], wavelet=wavelet, level=N)
    sizes = [c.shape[0] for c in coeffs]

    A = np.zeros((sum(sizes), n))

    for ii in range(n):
        coeffs = pywt.wavedec(Y[:, ii], wavelet=wavelet, level=N)
        A[:, ii] = np.concatenate(coeffs)

    # ----- mean coefficients and centered coefficients -----
    mu_A = A.mean(axis=1, keepdims=True)
    C = A - mu_A

    # ----- inverse wavelet transform of mean function -----
    mu_hat = pywt.waverec(
        pywt.array_to_coeffs(mu_A[:, 0], coeffs, output_format='wavedec'),
        wavelet
    )

    # ----- reconstruct eigenfunctions -----
    H = np.zeros((nt, d0))
    for k in range(d0):
        H[:, k] = pywt.waverec(
            pywt.array_to_coeffs(B[:, k], coeffs, output_format='wavedec'),
            wavelet
        )

    # ----- reconstruct curves -----
    Yhat = np.zeros_like(Y)
    scores = B.T @ C  # (d0, n)

    for t in range(n):
        Yhat[:, t] = mu_hat + H @ scores[:, t]

    # ----- residuals -----
    mEps_hat = Y - Yhat

    # ===============================
    # Bootstrap loop
    # ===============================
    for jj in range(NREP):

        Yboot = np.zeros_like(Y)
        Aboot = np.zeros_like(A)

        # resample residuals
        for ii in range(n):
            idx = np.random.randint(0, n)
            Yboot[:, ii] = Yhat[:, ii] + mEps_hat[:, idx]

            coeffs = pywt.wavedec(Yboot[:, ii], wavelet=wavelet, level=N)
            Aboot[:, ii] = np.concatenate(coeffs)

        # ----- center bootstrap coefficients -----
        mu_Aboot = Aboot.mean(axis=1, keepdims=True)
        Cb = Aboot - mu_Aboot

        # ----- lagged covariance operator -----
        C1 = Cb[:, :n - p]
        D1 = np.zeros((n - p, n - p))

        for k in range(1, p + 1):
            Ct = Cb[:, k:(n - p + k)]
            D1 += Ct.T @ Ct

        Dboot = C1 @ D1 @ C1.T / ((n - p) ** 2)

        # ----- eigendecomposition -----
        Lboot, _ = np.linalg.eig(Dboot)
        Lboot = np.sort(Lboot)[::-1]  # descending order

        vd0p1_boot[jj] = Lboot[d0]

    return vd0p1_boot
